# XGB Classifier with Wrapper-Based FS (Lexical + Content)

Steven Sison | March 9, 2024

## Description

This document will be used to train a model using the reduced feature set obtain by using the wrapper-based method, forward feature selection. The model will be evaluated in terms of the usual metrics (accuracy, precision, F1-score, recall) as well as the training time. The model will also be stored for future evaluation purposes.

## Training the Model

### Preliminaries

#### 1. Loading the Dataset

In [1]:
import pandas as pd                     # For data transformation
import numpy as numpy                   # For scientific calculations
import seaborn as sns                   # For data visualizations
import matplotlib.pyplot as plt         # For plotting
import plotly.graph_objects as go       # For plotting
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, ConfusionMatrixDisplay
from xgboost import XGBClassifier, DMatrix, train
from sklearn.pipeline import Pipeline
import time
from datetime import datetime
import joblib
import os

dataset = pd.read_csv("../../../02_feature-engineering/final-datasets/updated_binary_unbalanced_withContent.csv")   

dataset.head()



,url,url_type,blank_lines_count,blank_spaces_count,word_count,average_word_len,webpage_size,webpage_entropy,js_count,sus_js_count,...,has_cgi_in_string,has_php_in_string,has_abuse_in_string,has_admin_in_string,has_bin_in_string,has_personal_in_string,has_update_in_string,has_verification_in_string,url_scheme,openpagerank
0,br-icloud.com.br,1,14,36,0,0.000000,1186,5.269303,2,0,...,0,0,0,0,0,0,0,0,0,10000001
1,bopsecrets.org/rexroth/cr/1.htm,0,415,3952,11450,4.683930,26155,4.565537,0,0,...,0,0,0,0,0,0,0,0,0,69061
2,allmusic.com/album/crazy-from-the-heat-r16990,0,5715,88450,420,5.833333,339327,4.656704,13,10,...,0,0,0,0,0,0,0,0,0,9043
3,myspace.com/video/vid/30602581,0,1461,6292,3049,4.515907,89249,5.149580,11,2,...,0,0,0,0,0,0,0,0,0,104
4,uk.linkedin.com/pub/steve-rubenstein/8/718/755,0,43,273,0,0.000000,1530,4.754726,1,1,...,0,0,0,0,0,0,0,0,0,8


In [2]:
dataset.head()

,url,url_type,blank_lines_count,blank_spaces_count,word_count,average_word_len,webpage_size,webpage_entropy,js_count,sus_js_count,...,has_cgi_in_string,has_php_in_string,has_abuse_in_string,has_admin_in_string,has_bin_in_string,has_personal_in_string,has_update_in_string,has_verification_in_string,url_scheme,openpagerank
0,br-icloud.com.br,1,14,36,0,0.000000,1186,5.269303,2,0,...,0,0,0,0,0,0,0,0,0,10000001
1,bopsecrets.org/rexroth/cr/1.htm,0,415,3952,11450,4.683930,26155,4.565537,0,0,...,0,0,0,0,0,0,0,0,0,69061
2,allmusic.com/album/crazy-from-the-heat-r16990,0,5715,88450,420,5.833333,339327,4.656704,13,10,...,0,0,0,0,0,0,0,0,0,9043
3,myspace.com/video/vid/30602581,0,1461,6292,3049,4.515907,89249,5.149580,11,2,...,0,0,0,0,0,0,0,0,0,104
4,uk.linkedin.com/pub/steve-rubenstein/8/718/755,0,43,273,0,0.000000,1530,4.754726,1,1,...,0,0,0,0,0,0,0,0,0,8


In [3]:
from sklearn.utils import resample

dataset_benign = dataset[(dataset['url_type'] == 0)]
dataset_malicious = dataset[(dataset['url_type'] == 1)]

dataset_malicious_upsampled = resample(dataset_malicious,
                                        replace=True,
                                        n_samples = dataset_benign.shape[0],
                                        random_state = 15)

dataset_upsampled = pd.concat([dataset_benign, dataset_malicious_upsampled])

dataset_upsampled['url_type'].value_counts()

url_type
0    120080
1    120080
Name: count, dtype: int64

In [4]:
x_train, x_test, y_train, y_test = train_test_split(dataset.drop(columns=['url_type']), dataset['url_type'], test_size = 0.2, random_state=42)

#### 2. Preprocessing (Balancing)

In [5]:
dataset_upsampled['url_type'].value_counts()

url_type
0    120080
1    120080
Name: count, dtype: int64

#### 3. Removing Unnecessary Features

In [6]:
important_features_wrapper_25 = ['url_length',
                                'url_domain_entropy',
                                'url_is_digits_in_domain',
                                'url_number_of_parameters',
                                'url_number_of_digits',
                                'url_string_entropy',
                                'url_path_length',
                                'url_host_length',
                                'url_domain_len',
                                'url_num_subdomain',
                                'url_number_of_fragments',
                                'url_is_encoded',
                                'url_number_of_letters',
                                'url_num_periods',
                                'url_num_of_hyphens',
                                'url_num_underscore',
                                'url_num_forward_slash',
                                'url_num_semicolon',
                                'url_num_mod_sign',
                                'has_login_in_string',
                                'has_signin_in_string',
                                'has_logon_in_string',
                                'has_loginasp_in_string',
                                'has_exe_in_string',
                                'has_viewerphp_in_string',
                                'has_getImageasp_in_string',
                                'has_paypal_in_string',
                                'has_dbsysphp_in_string',
                                'has_shopping_in_string',
                                'has_php_in_string',
                                'has_bin_in_string',
                                'has_personal_in_string',
                                'blank_lines_count', 
                                'word_count', 
                                'js_count', 
                                'js_link_count', 
                                'js_winopen_count', 
                                'meta_tag_count', 
                                'anchor_tag_count', 
                                'input_tag_count', 
                                'has_free_in_html',
                                'openpagerank']

important_features_wrapper_25_lexical = ['blank_lines_count',
                                         'url_length', 
                                         'url_domain_entropy', 
                                         'url_number_of_digits', 
                                         'url_path_length', 
                                         'url_host_length', 
                                         'url_number_of_subdirectories', 
                                         'url_has_port', 
                                         'url_number_of_fragments', 
                                         'url_num_periods',
                                         'has_ebayisapi_in_string', 
                                         'has_logon_in_string', 
                                         'has_signon_in_string', 
                                         'has_loginasp_in_string', 
                                         'has_exe_in_string', 
                                         'has_zip_in_string', 
                                         'has_paypal_in_string']

important_features_wrapper_25_content = ['blank_lines_count', 
                                         'word_count', 
                                         'js_count', 
                                         'js_link_count', 
                                         'js_winopen_count', 
                                         'meta_tag_count', 
                                         'anchor_tag_count', 
                                         'input_tag_count', 
                                         'has_free_in_html',
                                         'openpagerank'] 


important_features_wrapper_65 = ['url_length',
                                'url_domain_entropy',
                                'url_is_digits_in_domain',
                                'url_number_of_parameters',
                                'url_number_of_digits',
                                'url_string_entropy',
                                'url_path_length',
                                'url_host_length',
                                'url_domain_len',
                                'url_num_subdomain',
                                'url_number_of_fragments',
                                'url_is_encoded',
                                'url_number_of_letters',
                                'url_num_periods',
                                'url_num_of_hyphens',
                                'url_num_underscore',
                                'url_num_forward_slash',
                                'url_num_semicolon',
                                'url_num_mod_sign',
                                'has_login_in_string',
                                'has_signin_in_string',
                                'has_logon_in_string',
                                'has_loginasp_in_string',
                                'has_exe_in_string',
                                'has_viewerphp_in_string',
                                'has_getImageasp_in_string',
                                'has_paypal_in_string',
                                'has_dbsysphp_in_string',
                                'has_shopping_in_string',
                                'has_php_in_string',
                                'has_bin_in_string',
                                'has_personal_in_string',
                                'blank_lines_count', 
                                'word_count', 
                                'js_count', 
                                'js_find_count', 
                                'js_link_count', 
                                'js_winopen_count', 
                                'title_tag_presence', 
                                'meta_tag_count', 
                                'anchor_tag_count', 
                                'applet_tag_count', 
                                'input_tag_count', 
                                'has_free_in_html', 
                                'has_access_in_html',
                                'openpagerank']

important_features_wrapper_65_lexical = ['url_length', 
                                         'url_ip_in_domain', 
                                         'url_domain_entropy', 
                                         'url_is_digits_in_domain', 
                                         'url_number_of_digits', 
                                         'url_string_entropy', 
                                         'url_path_length', 
                                         'url_host_length', 
                                         'url_number_of_subdirectories', 
                                         'url_num_subdomain', 
                                         'url_has_port', 
                                         'url_number_of_fragments', 
                                         'url_num_periods', 
                                         'url_num_equal', 
                                         'url_num_open_parenthesis', 
                                         'url_num_close_parenthesis', 
                                         'url_num_ampersand', 
                                         'url_num_at', 
                                         'has_account_in_string', 
                                         'has_webscr_in_string', 
                                         'has_ebayisapi_in_string', 
                                         'has_signin_in_string', 
                                         'has_banking_in_string', 
                                         'has_confirm_in_string', 
                                         'has_logon_in_string', 
                                         'has_signon_in_string', 
                                         'has_loginasp_in_string', 
                                         'has_loginphp_in_string', 
                                         'has_exe_in_string', 
                                         'has_zip_in_string', 
                                         'has_rar_in_string', 
                                         'has_jpg_in_string', 
                                         'has_gif_in_string', 
                                         'has_viewerphp_in_string', 
                                         'has_getImageasp_in_string', 
                                         'has_plugins_in_string', 
                                         'has_paypal_in_string', 
                                         'has_dbsysphp_in_string', 
                                         'has_configbin_in_string', 
                                         'has_downloadphp_in_string', 
                                         'has_payment_in_string', 
                                         'has_files_in_string', 
                                         'has_shopping_in_string', 
                                         'has_mailphp_in_string', 
                                         'has_jar_in_string', 
                                         'has_swf_in_string', 
                                         'has_cgi_in_string', 
                                         'has_php_in_string', 
                                         'has_abuse_in_string', 
                                         'has_bin_in_string', 
                                         'has_update_in_string', 
                                         'has_verification_in_string'] 

important_features_wrapper_65_content = ['blank_lines_count', 
                                         'word_count', 
                                         'js_count', 
                                         'js_find_count', 
                                         'js_link_count', 
                                         'js_winopen_count', 
                                         'title_tag_presence', 
                                         'meta_tag_count', 
                                         'anchor_tag_count', 
                                         'applet_tag_count', 
                                         'input_tag_count', 
                                         'has_free_in_html', 
                                         'has_access_in_html',
                                         'openpagerank'] 


X_test_25 = x_test[important_features_wrapper_25]
X_train_25 = x_train[important_features_wrapper_25]

X_test_65 = x_test[important_features_wrapper_65]
X_train_65 = x_train[important_features_wrapper_65]

#### Hyper-parameter Tuning

##### 25 Features

In [7]:
import optuna
from sklearn.metrics import mean_squared_error # or any other metric
from sklearn.model_selection import train_test_split

c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Define the objective function for Optuna
def objective_25(trial):
    # Define the search space for hyperparameters
    param = {
        'objective': 'binary:hinge',
        'eval_metric': 'error',
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'n_estimators': 100000, # Fix the boosting round and use early stopping
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 10.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),
        'lambda': trial.suggest_float('lambda', 0.1, 10.0),
        'alpha': trial.suggest_float('alpha', 0.0, 10.0),
    }
    
    # Split the data into further training and validation sets (three sets are preferable)
    train_data, valid_data, train_target, valid_target = train_test_split(X_train_25, y_train, test_size=0.2, random_state=42)
    
    # Convert the data into DMatrix format
    dtrain = DMatrix(train_data, label=train_target)
    dvalid = DMatrix(valid_data, label=valid_target)
    
    # Define the pruning callback for early stopping
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-error')
    
    # Train the model with early stopping
    model = train(param, dtrain, num_boost_round=100000, evals=[(dvalid, 'validation')], early_stopping_rounds=100, callbacks=[pruning_callback])
    
    # Make predictions on the test set
    dtest = DMatrix(valid_data)
    y_pred = model.predict(dtest)
    
    # Calculate the root mean squared error
    error = mean_squared_error(valid_target, y_pred, squared=False)
    
    return error

# Create an Optuna study and optimize the objective function
study_25 = optuna.create_study(direction='minimize')
study_25.optimize(objective_25, n_trials=100) # Control the number of trials

# Print the best hyperparameters and the best RMSE
best_params_25 = study_25.best_params
best_error_25 = study_25.best_value
print("Best Hyperparameters (25 Features): ", best_params_25)
print("Best Error (25 Features): ", best_error_25)

[I 2024-05-14 16:10:51,922] A new study created in memory with name: no-name-66829208-418c-4ea6-81cc-f19195bbb164


[0]	validation-error:0.65995
[1]	validation-error:0.65995
[2]	validation-error:0.65995
[3]	validation-error:0.65995
[4]	validation-error:0.65995
[5]	validation-error:0.65995
[6]	validation-error:0.65995
[7]	validation-error:0.65995
[8]	validation-error:0.44119
[9]	validation-error:0.25467
[10]	validation-error:0.16874
[11]	validation-error:0.12917
[12]	validation-error:0.10248
[13]	validation-error:0.08579


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [16:10:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[14]	validation-error:0.07431
[15]	validation-error:0.06619
[16]	validation-error:0.06215
[17]	validation-error:0.05955
[18]	validation-error:0.05626
[19]	validation-error:0.05465
[20]	validation-error:0.05269
[21]	validation-error:0.05239
[22]	validation-error:0.05115
[23]	validation-error:0.05167
[24]	validation-error:0.05074
[25]	validation-error:0.05016
[26]	validation-error:0.05030
[27]	validation-error:0.04917
[28]	validation-error:0.04906
[29]	validation-error:0.04841
[30]	validation-error:0.04845
[31]	validation-error:0.04810
[32]	validation-error:0.04776
[33]	validation-error:0.04749
[34]	validation-error:0.04677
[35]	validation-error:0.04649
[36]	validation-error:0.04622
[37]	validation-error:0.04625
[38]	validation-error:0.04564
[39]	validation-error:0.04526
[40]	validation-error:0.04523
[41]	validation-error:0.04505
[42]	validation-error:0.04475
[43]	validation-error:0.04502
[44]	validation-error:0.04437
[45]	validation-error:0.04413
[46]	validation-error:0.04403
[47]	valid

[I 2024-05-14 16:10:56,600] Trial 0 finished with value: 0.19580343851705273 and parameters: {'eta': 0.06299149943507149, 'max_depth': 9, 'subsample': 0.5066276975870831, 'colsample_bytree': 0.7016182912059428, 'gamma': 6.055046816185295, 'min_child_weight': 5.906966035216769, 'lambda': 6.990298245635983, 'alpha': 4.401576810997417}. Best is trial 0 with value: 0.19580343851705273.
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [16:10:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


...

[0]	validation-error:0.65995
[1]	validation-error:0.65995
[2]	validation-error:0.15007
[3]	validation-error:0.10361
[4]	validation-error:0.08840
[5]	validation-error:0.07880


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [16:13:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-05-14 16:13:17,715] Trial 98 pruned. Trial was pruned at iteration 6.


[0]	validation-error:0.65995
[1]	validation-error:0.65995
[2]	validation-error:0.65995


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [16:13:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-05-14 16:13:18,020] Trial 99 pruned. Trial was pruned at iteration 2.


Best Hyperparameters (25 Features):  {'eta': 0.03603714520052025, 'max_depth': 6, 'subsample': 0.5783404347330316, 'colsample_bytree': 0.8613263389287014, 'gamma': 0.20027152461942976, 'min_child_weight': 9.951867505510057, 'lambda': 5.47931282954197, 'alpha': 1.4835353267877427}
Best Error (25 Features):  0.1754055938360889


##### 33 Features (Purely Lexical)

In [9]:
# Define the objective function for Optuna
def objective_65(trial):
    # Define the search space for hyperparameters
    param = {
        'objective': 'binary:hinge',
        'eval_metric': 'error',
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'n_estimators': 100000, # Fix the boosting round and use early stopping
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 10.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),
        'lambda': trial.suggest_float('lambda', 0.1, 10.0),
        'alpha': trial.suggest_float('alpha', 0.0, 10.0),
    }
    
    # Split the data into further training and validation sets (three sets are preferable)
    train_data, valid_data, train_target, valid_target = train_test_split(X_train_65, y_train, test_size=0.2, random_state=42)
    
    # Convert the data into DMatrix format
    dtrain = DMatrix(train_data, label=train_target)
    dvalid = DMatrix(valid_data, label=valid_target)
    
    # Define the pruning callback for early stopping
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-error')
    
    # Train the model with early stopping
    model = train(param, dtrain, num_boost_round=100000, evals=[(dvalid, 'validation')], early_stopping_rounds=100, callbacks=[pruning_callback])
    
    # Make predictions on the test set
    dtest = DMatrix(valid_data)
    y_pred = model.predict(dtest)
    
    # Calculate the root mean squared error
    error = mean_squared_error(valid_target, y_pred, squared=False)
    
    return error

# Create an Optuna study and optimize the objective function
study_65 = optuna.create_study(direction='minimize')
study_65.optimize(objective_65, n_trials=100) # Control the number of trials

# Print the best hyperparameters and the best RMSE
best_params_65 = study_65.best_params
best_error_65 = study_65.best_value
print("Best Hyperparameters (60 Features): ", best_params_65)
print("Best Error (60 Features): ", best_error_65)

[I 2024-05-14 16:13:18,042] A new study created in memory with name: no-name-f53fdb89-7711-4294-b2a3-915fb565d2fe


[0]	validation-error:0.65995
[1]	validation-error:0.23610
[2]	validation-error:0.08733
[3]	validation-error:0.06633
[4]	validation-error:0.06027
[5]	validation-error:0.05660
[6]	validation-error:0.05283
[7]	validation-error:0.05201
[8]	validation-error:0.05050
[9]	validation-error:0.04923
[10]	validation-error:0.04851


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [16:13:18] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[11]	validation-error:0.04814
[12]	validation-error:0.04684
[13]	validation-error:0.04632
[14]	validation-error:0.04553
[15]	validation-error:0.04471
[16]	validation-error:0.04362
[17]	validation-error:0.04317
[18]	validation-error:0.04218
[19]	validation-error:0.04156
[20]	validation-error:0.04111
[21]	validation-error:0.04108
[22]	validation-error:0.04053
[23]	validation-error:0.03971
[24]	validation-error:0.03920
[25]	validation-error:0.03882
[26]	validation-error:0.03920
[27]	validation-error:0.03834
[28]	validation-error:0.03830
[29]	validation-error:0.03765
[30]	validation-error:0.03731
[31]	validation-error:0.03711
[32]	validation-error:0.03714
[33]	validation-error:0.03711
[34]	validation-error:0.03669
[35]	validation-error:0.03659
[36]	validation-error:0.03656
[37]	validation-error:0.03666
[38]	validation-error:0.03632
[39]	validation-error:0.03621
[40]	validation-error:0.03635
[41]	validation-error:0.03635
[42]	validation-error:0.03642
[43]	validation-error:0.03635
[44]	valid

[I 2024-05-14 16:13:20,857] Trial 0 finished with value: 0.1821138390450897 and parameters: {'eta': 0.2985446023744014, 'max_depth': 8, 'subsample': 0.9468663747082642, 'colsample_bytree': 0.84915292981166, 'gamma': 3.7784543429760378, 'min_child_weight': 9.779980294038317, 'lambda': 1.8064750465858752, 'alpha': 2.597547518088846}. Best is trial 0 with value: 0.1821138390450897.
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [16:13:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


...

[10]	validation-error:0.04242
[11]	validation-error:0.04180
[12]	validation-error:0.04087
[13]	validation-error:0.04002
[14]	validation-error:0.03968
[15]	validation-error:0.03947
[16]	validation-error:0.03902
[17]	validation-error:0.03906
[18]	validation-error:0.03841
[19]	validation-error:0.03817
[20]	validation-error:0.03772
[21]	validation-error:0.03752
[22]	validation-error:0.03683
[23]	validation-error:0.03673
[24]	validation-error:0.03639
[25]	validation-error:0.03601
[26]	validation-error:0.03611
[27]	validation-error:0.03594
[28]	validation-error:0.03553
[29]	validation-error:0.03560
[30]	validation-error:0.03560
[31]	validation-error:0.03529


[I 2024-05-14 16:14:42,963] Trial 98 pruned. Trial was pruned at iteration 31.


[0]	validation-error:0.65995


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [16:14:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-05-14 16:14:43,234] Trial 99 pruned. Trial was pruned at iteration 1.


Best Hyperparameters (60 Features):  {'eta': 0.0695957757018085, 'max_depth': 10, 'subsample': 0.6204690124093536, 'colsample_bytree': 0.5743483693964326, 'gamma': 1.1329910665388132, 'min_child_weight': 1.594127365915223, 'lambda': 8.0295706254282, 'alpha': 0.43234925993634543}
Best Error (60 Features):  0.17324364956115884


In [10]:
# print(best_params_25)
print(best_params_65)

{'eta': 0.0695957757018085, 'max_depth': 10, 'subsample': 0.6204690124093536, 'colsample_bytree': 0.5743483693964326, 'gamma': 1.1329910665388132, 'min_child_weight': 1.594127365915223, 'lambda': 8.0295706254282, 'alpha': 0.43234925993634543}


#### Model Training

In [11]:
from sklearn.model_selection import KFold
import optuna

# Initialize CV
cv = KFold(n_splits=10, shuffle=True, random_state=1)

# best_params_25['objective'] = 'binary:hinge'
# best_params_25['eval_metric'] = 'error'

best_params_65['objective'] = 'binary:hinge'
best_params_65['eval_metric'] = 'error'

# Convert the data into DMatrix format
# dtrain_25 = DMatrix(X_train_25, label=y_train)
# dvalid_25 = DMatrix(X_test_25, label=y_test)

dtrain_65 = DMatrix(X_train_65, label=y_train)
dvalid_65 = DMatrix(X_test_65, label=y_test)

# Train the Model
# xgb_classifier_25 = train(best_params_25, dtrain_25, num_boost_round=3000)
# y_pred_25 = xgb_classifier_25.predict(dvalid_25)

# print("Model with 12 Features Done.")

xgb_classifier_65 = train(best_params_65, dtrain_65, num_boost_round=3000)
y_pred_65 = xgb_classifier_65.predict(dvalid_65)

print("Model with 33 Features Done.")

Model with 33 Features Done.


#### Evaluation

In [12]:
# Classification Report
# print(classification_report(y_test, y_pred_25))
print(classification_report(y_test, y_pred_65))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97     23942
           1       0.96      0.94      0.95     12541

    accuracy                           0.97     36483
   macro avg       0.96      0.96      0.96     36483
weighted avg       0.97      0.97      0.97     36483



In [13]:
'''# Confusion Matrix for 12 Features
cm_up = confusion_matrix(y_test, y_pred_12, labels=xgb_classifier_12.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm_up, display_labels = xgb_classifier_12.classes_)
disp.plot()
plt.show()'''

'# Confusion Matrix for 12 Features\ncm_up = confusion_matrix(y_test, y_pred_12, labels=xgb_classifier_12.classes_)\ndisp = ConfusionMatrixDisplay(confusion_matrix = cm_up, display_labels = xgb_classifier_12.classes_)\ndisp.plot()\nplt.show()'

In [14]:
'''# Cross Validation Score
scores = cross_val_score(XGBClassifier(**params_gbm),
                        X_train, y_train, scoring='accuracy', cv=cv).mean()

print(scores)'''

"# Cross Validation Score\nscores = cross_val_score(XGBClassifier(**params_gbm),\n                        X_train, y_train, scoring='accuracy', cv=cv).mean()\n\nprint(scores)"

In [15]:
# Dumping the model
# joblib.dump(xgb_classifier_25, 'xgb_wrapper_25_lexical-content.sav')
# joblib.dump(xgb_classifier_65, 'xgb_wrapper_65_lexical-content.sav')

In [18]:
import content_generator_25
import content_generator_65
import time

'''def xgb_predict_maliciousness_25(url):

    numerical_values = content_generator_25.feature_generator(url)
    # print(numerical_values)
    numerical_values = DMatrix(numerical_values)

    match xgb_classifier_25.predict(numerical_values):
        case 0:
            return "Benign"
        case 1:
            return "Malware"
        case 2:
            return "Phishing"
        case 3:
            return "Defacement"'''
        
def xgb_predict_maliciousness_65(url):

    numerical_values = content_generator_65.feature_generator(url)
    # print(numerical_values)
    numerical_values = DMatrix(numerical_values)

    match xgb_classifier_65.predict(numerical_values):
        case 0:
            return "Benign"
        case 1:
            return "Malware"
        case 2:
            return "Phishing"
        case 3:
            return "Defacement"

url = "online.bdo.com.ph/"
print("Current URL: "+url)

'''print("------------- Wrapper-Based (25 Features) -------------")
for i in range(1):
    start = time.perf_counter()
    prediction = xgb_predict_maliciousness_25(url)
    end = time.perf_counter()
    print("Trial "+str(i))
    print(prediction)
    print(end-start)'''

print("------------- Wrapper-Based (65 Features) -------------")
for i in range(1):
    start = time.perf_counter()
    prediction = xgb_predict_maliciousness_65(url)
    end = time.perf_counter()
    print("Trial "+str(i))
    print(prediction)
    print(end-start)

Current URL: online.bdo.com.ph/
------------- Wrapper-Based (65 Features) -------------


TypeError: The source html cannot be fetched.